In [1]:
!pip install pennylane

In [2]:
import pennylane as qml
from pennylane import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [3]:
def potential_energy_surface(symbols, bond_lengths):
    """Calculates the molecular energy over various bond lengths (AKA the
    potential energy surface) using the Hartree Fock method.

    Args:
        symbols (list(string)):
            A list of atomic symbols that comprise the diatomic molecule of interest.
        bond_lengths (numpy.tensor): Bond lengths to calculate the energy over.


    Returns:
        hf_energies (numpy.tensor):
            The Hartree Fock energies at every bond length value.
    """

    hf_energies = []


    # Put your code here #

    geometry = np.array([
        [0.0, 0.0, -bond_lengths/2], # First nuclear position
        [0.0, 0.0,  bond_lengths/2]], # Second nuclear position
        requires_grad=True
    )

    mol = qml.qchem.Molecule(symbols,geometry) #,alpha=alpha,coeff=coeffs)
    cost = qml.qchem.hf_energy(mol)
    hf_energies.append(
        cost(geometry)
    )

    return np.array(hf_energies)
def ground_energy(hf_energies):
    """Finds the minimum energy of a molecule given its potential energy surface.

    Args:
        hf_energies (numpy.tensor):

    Returns:
        (float): The minumum energy in units of hartrees.
    """

    ind = np.argmin(hf_energies)
    return hf_energies[ind]

In [4]:
# Generating Ground State using HF Approach

symbols = ["H", "H"]
geometry = np.array([0.0, 0.0, -0.6614, 0.0, 0.0, 0.6614])
electrons = 2

H, qubits = qml.qchem.molecular_hamiltonian(symbols, geometry)
print("Number of qubits = ", qubits)
print(H)

# Estimating determinant
hf = qml.qchem.hf_state(electrons, qubits)
print(hf)

ground_state_energy = ground_energy(potential_energy_surface(symbols, 2*0.6614))

Number of qubits =  4
  (-0.2427450126094144) [Z2]
+ (-0.2427450126094144) [Z3]
+ (-0.042072551947439224) [I0]
+ (0.1777135822909176) [Z0]
+ (0.1777135822909176) [Z1]
+ (0.12293330449299361) [Z0 Z2]
+ (0.12293330449299361) [Z1 Z3]
+ (0.16768338855601356) [Z0 Z3]
+ (0.16768338855601356) [Z1 Z2]
+ (0.17059759276836803) [Z0 Z1]
+ (0.1762766139418181) [Z2 Z3]
+ (-0.044750084063019925) [Y0 Y1 X2 X3]
+ (-0.044750084063019925) [X0 X1 Y2 Y3]
+ (0.044750084063019925) [Y0 X1 X2 Y3]
+ (0.044750084063019925) [X0 Y1 Y2 X3]
[1 1 0 0]


In [5]:
# Generating single and double excitations for trial wavefunction, again using HF
singles, doubles = qml.qchem.excitations(electrons, qubits)

In [6]:
# Building Trial Wave Function
dev = qml.device("default.qubit", wires=qubits)

@qml.qnode(dev)
def ansatz(params, wires, det, H, singles=singles, doubles=doubles):
  # Initialising circuit with determinant (estimated using HF)
  qml.AllSinglesDoubles(params[:len(singles) + len(doubles)], wires=wires, hf_state=det, singles=singles, doubles=doubles)

  rotation_params = params[len(singles) + len(doubles):]

  # Now performing Jastrow Function
  for i in range(len(wires)):
        qml.RX(rotation_params[i], wires=i)
        qml.RY(rotation_params[len(wires) + i], wires=i)
  return qml.expval(H)

In [7]:
def excited_energy(params, wires, det, H, final=False, perturbation=True):
  """# First calculate ground state energy
  ground_energy = ansatz(params, wires, det, H)"""
  # Perturb params for electron movement

  perturbed_params = params + np.random.normal(scale=0.1, size=params.shape)

  if perturbation:
    excited_energy = ansatz(perturbed_params, wires, det, H)
  else:
    excited_energy = ansatz(params, wires, det, H)
  if not final:
      excited_energy = float(excited_energy._value)
  else:
      excited_energy = float(excited_energy)

  return excited_energy

In [ ]:
# QMC Simulation

# Initialize parameters for the ansatz
n_params = qubits * 2
params = np.random.normal(0, np.pi, len(singles) + len(doubles) + n_params, requires_grad=True)


#Initalizing parameters for training progress
n_iterations = 1000
excited_energies = []
x = np.arange(n_iterations, requires_grad=False)
pbar = tqdm(total=n_iterations)
best_cost = float('inf')

# Choose an optimizer
optimizer = qml.AdamOptimizer(stepsize=0.1)

# Optimize the parameters
print("Beginning training")
for it in range(n_iterations):
    params, cost = optimizer.step_and_cost(lambda p: excited_energy(p, wires=range(qubits), det=hf, H=H), params)
    excited_energies.append(cost)
    if cost < best_cost:
        best_params = params
    if it % (n_iterations / 20) == 0 and it != 0:
      pbar.update(n_iterations / 20)
      print("\n")
      print(params)
      print("\n iteration: ", it, " cost: ", cost)

print("Training over")
first_excited_state_energy = excited_energy(params, wires=range(qubits), det=hf, H=H, final=True, perturbation=False)
print(first_excited_state_energy)
print(ground_state_energy)
energy_gap = first_excited_state_energy - ground_state_energy
print(energy_gap)

  0%|          | 0/1000 [00:00<?, ?it/s]

Beginning training


/usr/local/lib/python3.10/dist-packages/autograd/tracer.py:14: UserWarning: Output seems independent of input.
  warnings.warn("Output seems independent of input.")
  5%|▌         | 50.0/1000 [00:02<00:43, 21.62it/s]



[ 0.74244294 -2.81297281 -2.34329506 -0.27838103  2.21290408 -3.66504474
  3.35851468  2.25492055 -0.14111107 -3.01838717  6.35424824]

 iteration:  50  cost:  0.23447715406673733


 10%|█         | 100.0/1000 [00:05<00:55, 16.25it/s]

...



[ 0.74244294 -2.81297281 -2.34329506 -0.27838103  2.21290408 -3.66504474
  3.35851468  2.25492055 -0.14111107 -3.01838717  6.35424824]

 iteration:  500  cost:  0.34376053151509983


 55%|█████▌    | 550.0/1000 [00:31<00:19, 22.72it/s]



[ 0.74244294 -2.81297281 -2.34329506 -0.27838103  2.21290408 -3.66504474
  3.35851468  2.25492055 -0.14111107 -3.01838717  6.35424824]

 iteration:  550  cost:  0.3407019344526468


 60%|██████    | 600.0/1000 [00:33<00:16, 23.83it/s]



[ 0.74244294 -2.81297281 -2.34329506 -0.27838103  2.21290408 -3.66504474
  3.35851468  2.25492055 -0.14111107 -3.01838717  6.35424824]

 iteration:  600  cost:  0.32258972472425734


In [ ]:
plt.plot(x, excited_energies)
plt.show()

In [ ]:
dev = qml.device("default.qubit", wires=qubits, shots=10000)

@qml.qnode(dev)
def orbitals(params, wires, det, H):
    qml.AllSinglesDoubles(params[:len(singles) + len(doubles)], wires=wires, hf_state=det, singles=singles, doubles=doubles)

    rotation_params = params[len(singles) + len(doubles):]

  # Now performing Jastrow Function
    n_layers = len(params) // len(wires) // 2
    for _ in range(n_layers):
        for i in range(len(wires)):
            qml.RX(rotation_params[i], wires=i)
            qml.RY(rotation_params[len(wires) + i], wires=i)
    """for i in range(0, len(wires), 2):
        qml.CRX(rotation_params[i], wires=[i, i+1])"""
    return qml.counts()

orbitals(params, wires=range(qubits), det=hf, H=H)